In [2]:
# the imports
from pymongo import MongoClient
from pprint import pprint

In [3]:
client = MongoClient('localhost:27017')
db = client.venmo
transactions = db.venmo

In [4]:
transactions.count_documents({}) 

7076585

In [5]:
# one random sample from the dataset
sampled_transactions = transactions.aggregate([{ '$sample' : {'size' : 1}}])
for transaction in sampled_transactions:
    pprint(transaction.keys())

dict_keys(['_id', 'date_updated', 'transfer', 'app', 'comments', 'payment', 'note', 'audience', 'likes', 'mentions', 'date_created', 'type', 'id', 'authorization'])
